In [ ]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from lstm import lstm_model, train_lstm
from data_processing import get_and_save_data, get_train_data, convert_to_1d
from globals import DATA

In [ ]:
# If you do not have data saved locally, run this
get_and_save_data(dir_path=DATA)

In [ ]:
train_data, encoder_input, decoder_output = get_train_data(dir_path=DATA, batch_size=32)

In [ ]:
input_dim, hidden_dim, layer_dim, output_dim = 2, 64, 3, 1

model_untrained = lstm_model(input_dim, hidden_dim, layer_dim, output_dim)   #not trained, for comparison
model_trained = lstm_model(input_dim, hidden_dim, layer_dim, output_dim)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_trained.to(device)
model_untrained.to(device)

In [ ]:
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
loss_curve = train_lstm(model_trained, device, train_loader)

In [ ]:
output_trained = model_trained(torch.from_numpy(encoder_input[0:1280]).to(device).float())
output_untrained = model_untrained(torch.from_numpy(encoder_input[0:1280]).to(device).float())

plt.plot(output_trained.cpu().detach().numpy()[50,:,0], label='trained')
plt.plot(output_untrained.cpu().detach().numpy()[50,:,0], label='untrained')
plt.plot(decoder_output[50,:,0], label='ground truth')
plt.legend()

In [ ]:
output_trained_1d = convert_to_1d(output_trained.cpu().detach().numpy())